# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
weather_df = pd.read_csv("CityWeather.csv") 

weather_df.head()

,Unnamed: 0,City Name,Lat,Lng,Max_temp,Humidity,Cloudiness,Wind,Country,Date
0,0,vaini,-21.20,-175.20,68.000,100.0,40.0,3.60,TO,1.595512e+09
1,1,mount isa,-20.73,139.50,57.200,44.0,0.0,5.39,AU,1.595512e+09
2,2,georgetown,5.41,100.34,84.200,79.0,20.0,0.88,MY,1.595512e+09
3,3,dikson,73.51,80.55,42.494,86.0,0.0,4.61,RU,1.595512e+09
4,4,barra patuca,15.80,-84.28,82.148,77.0,94.0,2.12,HN,1.595512e+09


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
# Configure gmaps with API key
gmaps.configure(api_key=g_key)

In [4]:
# Store latitude and longitude in locations
locations = weather_df[["Lat", "Lng"]]
humidity = weather_df["Humidity"]


In [5]:
fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=100,
                                 point_radius = 3)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [6]:
narrow_df = weather_df.loc[(weather_df["Max_temp"]<80) & (weather_df["Max_temp"]>70)\
                           & (weather_df["Wind"]<10) & (weather_df["Cloudiness"] == 0)].dropna()
narrow_df

,Unnamed: 0,City Name,Lat,Lng,Max_temp,Humidity,Cloudiness,Wind,Country,Date
15,16,east london,-33.02,27.91,75.200,27.0,0.0,3.60,ZA,1.595512e+09
20,21,cape town,-33.93,18.42,75.992,56.0,0.0,2.10,ZA,1.595512e+09
65,73,hermanus,-34.42,19.23,71.996,58.0,0.0,0.45,ZA,1.595512e+09
72,81,bataipora,-22.30,-53.27,79.412,43.0,0.0,3.71,BR,1.595511e+09
85,98,merauke,-8.47,140.33,75.884,81.0,0.0,6.69,ID,1.595512e+09
137,153,chifeng,42.27,118.96,76.064,48.0,0.0,1.37,CN,1.595512e+09
150,169,kruisfontein,-34.00,24.73,73.508,37.0,0.0,2.31,ZA,1.595512e+09
172,194,port alfred,-33.59,26.89,71.420,40.0,0.0,3.33,ZA,1.595512e+09
190,212,saldanha,-33.01,17.94,78.800,36.0,0.0,1.00,ZA,1.595512e+09
195,217,ayagoz,47.97,80.44,73.130,34.0,0.0,2.38,KZ,1.595512e+09


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [7]:
hotel_df = narrow_df[["City Name", "Country", "Lat", "Lng"]].copy()
hotel_df["Hotel Name"]=""
params = {"radius": 5000,
          "types": "lodging",
          "key": g_key}
for index,row in hotel_df.iterrows():
    Lat = row["Lat"]
    Lng = row["Lng"]
    params["location"]=f"{Lat}, {Lng}"
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    address = requests.get(base_url,params=params).json()
    try:
        hotel_df.loc[index, "Hotel Name"]=address["results"][0]["name"]
    except(KeyError, IndexError):
        print("Location not found")
hotel_df




Location not found


,City Name,Country,Lat,Lng,Hotel Name
15,east london,ZA,-33.02,27.91,Tu Casa
20,cape town,ZA,-33.93,18.42,Southern Sun Waterfront Cape Town
65,hermanus,ZA,-34.42,19.23,Misty Waves Boutique Hotel
72,bataipora,BR,-22.30,-53.27,Hotel Vale do Ivinhema
85,merauke,ID,-8.47,140.33,
137,chifeng,CN,42.27,118.96,Junyu Tiancheng Hotel
150,kruisfontein,ZA,-34.00,24.73,Oyster Bay House Rental
172,port alfred,ZA,-33.59,26.89,The Halyards Hotel
190,saldanha,ZA,-33.01,17.94,Blue Bay Lodge
195,ayagoz,KZ,47.97,80.44,Gostinitsa Nur


In [8]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City Name}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [9]:
# Add marker layer ontop of heat map
marker_layer=gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)
# Display figure
fig

Figure(layout=FigureLayout(height='420px'))